# Baseline network + pruning + quantization

This notebook trains the baseline network with exact same architecture as the one in paper [Recognition of handwritten Latin characters with diacritics using CNN](https://journals.pan.pl/dlibra/publication/136210/edition/119099/content/bulletin-of-the-polish-academy-of-sciences-technical-sciences-recognition-of-handwritten-latin-characters-with-diacritics-using-cnn-lukasik-edyta-charytanowicz-malgorzata-milosz-marek-tokovarov-michail-kaczorowska-monika-czerwinski-dariusz-zientarski-tomasz-2021-69-no-1?language=en)

Model architecture description:

"The architecture of the concrete CNN is shown in Fig. 2.
The input is a 32x32 binarized matrix. The input is then prop-
agated through 12 adaptable layers. First come two convolu-
tional layers having 32 filters with the size of 3x3 and stride 1.
Secondly, the output of the convolutional layer is fed to the
ReLU function. The output is down-sampled using a max-pool-
ing operation with a 2x2 stride. Next, the dropout technique is
used with the coefficient 0.25. The four operations (two con-
volutions, nonlinearity, max-pooling, and dropout) are repeated,
using 64 filters for the convolutional layers. The output of the
last layer is then flattened and fed through a fully connected
layer with 256 neurons and ReLU nonlinearities, dropped out
with the 0.25 coefficient, and a final output layer is fully con-
nected with a Softmax activation function. The Adam optimizer
and the cross-entropy loss function were used in the network. 
The output is a probability distribution over 89 classes."


Additionally, this network was pruned and quantized after training.



After training, model is serialized and uploaded to W&B project.

In [1]:
#! pip install -q tensorflow-model-optimization

In [2]:
#! pip install -U tensorboard_plugin_profile

In [3]:
%load_ext tensorboard

In [4]:
import pathlib
import shutil
import datetime
import numpy as np
import wandb
import tensorflow as tf
import tensorflow_model_optimization as tfmot

from typing import List

def load_data(run, artifact_name = "phcd_paper_splits_tfds") -> List[tf.data.Dataset]:
    """
    Downloads datasets from a wandb artifact and loads them into a list of tf.data.Datasets.
    """

    artifact = run.use_artifact(f"master-thesis/{artifact_name}:latest")
    artifact_dir = pathlib.Path(
        f"./artifacts/{artifact.name.replace(':', '-')}"
    ).resolve()
    if not artifact_dir.exists():
        artifact_dir = artifact.download()
        artifact_dir = pathlib.Path(artifact_dir).resolve()

    # if tf.__version__ minor is less than 10, use
    # tf.data.experimental.load instead of tf.data.Dataset.load

    if int(tf.__version__.split(".")[1]) < 10:
        load_function = tf.data.experimental.load
    else:
        load_function = tf.data.Dataset.load
    
    output_list = []
    for split in ["train", "test", "val"]:
        ds = load_function(str(artifact_dir / split), compression="GZIP")
        output_list.append(ds)
    
    return output_list

def get_number_of_classes(ds: tf.data.Dataset) -> int:
    """
    Returns the number of classes in a dataset.
    """
    labels_iterator= ds.map(lambda x, y: y).as_numpy_iterator()
    labels = np.concatenate(list(labels_iterator))
    return len(np.unique(labels))

def get_number_of_examples(ds: tf.data.Dataset) -> int:
    """
    Returns the number of examples in a dataset.
    """
    return sum(1 for _ in ds)

def preprocess_dataset(ds: tf.data.Dataset, batch_size: int, cache: bool = True) -> tf.data.Dataset:
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))  # normalize
    ds = ds.unbatch().batch(batch_size)
    if cache:
        ds = ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

def calculate_model_size_on_disk(path: str) -> int:
    return pathlib.Path(path).stat().st_size    

def calculate_model_num_parameters(model: tf.keras.Model) -> int:
    return model.count_params()

def calculate_model_flops(summary) -> float:
    # from run.summary get GFLOPs or GFLOPS whichever is available
    if "GFLOPs" in summary.keys():
        return summary.get("GFLOPs")
    elif "GFLOPS" in summary.keys():
        return summary.get("GFLOPS")
    else:
        return 0

2023-01-29 17:17:36.228726: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Available devices: ", tf.config.list_physical_devices())

Num GPUs Available:  1
Available devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-01-29 17:17:39.693353: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-29 17:17:39.706352: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-29 17:17:39.706575: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.


In [6]:
defaults = dict(
    batch_size=32*2,
    epochs=20,    
    optimizer="adam"
)

model_name = "baseline-pruned-numbers"

artifact_name = "phcd_paper_splits_tfds"
run = wandb.init(project="master-thesis", job_type="training", name=model_name, config=defaults, tags=[artifact_name])
    
# hyperparameters
epochs = wandb.config.epochs
bs = wandb.config.batch_size

ds_train, ds_test, ds_val = load_data(run, artifact_name=artifact_name)

num_classes = get_number_of_classes(ds_val)

ds_train = preprocess_dataset(ds_train, batch_size=bs)
ds_val = preprocess_dataset(ds_val, batch_size=bs)
ds_test = preprocess_dataset(ds_test, batch_size=bs, cache=False)

'''
The architecture of the concrete CNN is shown in Fig. 2.
The input is a 32x32 binarized matrix. 
The input is then propagated through 12 adaptable layers. 
First come two convolutional layers having 32 filters with the size of 3x3 and stride 1.
Secondly, the output of the convolutional layer is fed to the
ReLU function. The output is down-sampled using a max-pool-
ing operation with a 2x2 stride. Next, the dropout technique is
used with the coefficient 0.25. The four operations (two con-
volutions, nonlinearity, max-pooling, and dropout) are repeated,
using 64 filters for the convolutional layers. The output of the
last layer is then flattened and fed through a fully connected
layer with 256 neurons and ReLU nonlinearities, dropped out
with the 0.25 coefficient, and a final output layer is fully con-
nected with a Softmax activation function. The Adam optimizer
and the cross-entropy loss function were used in the network
[24]. The output is a probability distribution over 89 classes.
'''

model = tf.keras.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=(32, 32, 1)),
        
        # 2 Convolutional layers with 32 filters, 3x3 size, and stride 1
        tf.keras.layers.Conv2D(32, (3, 3), strides=1, activation='relu'),
        tf.keras.layers.Conv2D(32, (3, 3), strides=1, activation='relu'),
        
        # Max-pooling operation with 2x2 stride
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
        # Dropout with coefficient 0.25
        tf.keras.layers.Dropout(0.25),
        
        # Repeat above 4 operations using 64 filters
        tf.keras.layers.Conv2D(64, (3, 3), strides=1, activation='relu'),
        tf.keras.layers.Conv2D(64, (3, 3), strides=1, activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
        tf.keras.layers.Dropout(0.25),
        
        # Flatten the output and feed through fully connected layer with 256 neurons
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),

        # Dropout with coefficient 0.25
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gratkadlafana. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact phcd_paper_splits_tfds:latest, 163.87MB. 27 files... 
wandb:   27 of 27 files downloaded.  
Done. 0:0:0.0
2023-01-29 17:17:43.580377: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-01-29 17:17:43.581375: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-29 17:17:43.581660: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-29 17:17:43.581880: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-29 17:17:44.682702: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-29 17:17:44.683364: I tensorflow/compile

In [7]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 64)        36928     
                                                        

In [8]:
'''
# TF profiler stats - FLOPS calculation?

ProfileOptionBuilder = tf.profiler.ProfileOptionBuilder

param_stats = tf.profiler.profile(
    tf.get_default_graph(),
    options=ProfileOptionBuilder.trainable_variables_parameter())

# Use code view to associate statistics with Python codes.
opts = ProfileOptionBuilder(
    ProfileOptionBuilder.trainable_variables_parameter()
    ).with_node_names(show_name_regexes=['.*my_code1.py.*', '.*my_code2.py.*']
    ).build()
param_stats = tf.profiler.profile(
    tf.get_default_graph(),
    cmd='code',
    options=opts)

# param_stats can be tensorflow.tfprof.GraphNodeProto or
# tensorflow.tfprof.MultiGraphNodeProto, depending on the view.
# Let's print the root below.
sys.stdout.write('total_params: %d\n' % param_stats.total_parameters)
'''

"\n# TF profiler stats - FLOPS calculation?\n\nProfileOptionBuilder = tf.profiler.ProfileOptionBuilder\n\nparam_stats = tf.profiler.profile(\n    tf.get_default_graph(),\n    options=ProfileOptionBuilder.trainable_variables_parameter())\n\n# Use code view to associate statistics with Python codes.\nopts = ProfileOptionBuilder(\n    ProfileOptionBuilder.trainable_variables_parameter()\n    ).with_node_names(show_name_regexes=['.*my_code1.py.*', '.*my_code2.py.*']\n    ).build()\nparam_stats = tf.profiler.profile(\n    tf.get_default_graph(),\n    cmd='code',\n    options=opts)\n\n# param_stats can be tensorflow.tfprof.GraphNodeProto or\n# tensorflow.tfprof.MultiGraphNodeProto, depending on the view.\n# Let's print the root below.\nsys.stdout.write('total_params: %d\n' % param_stats.total_parameters)\n"

In [ ]:
# save the best model
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=f"./artifacts/{model_name}.h5",
    save_weights_only=False,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, profile_batch='10, 20')

wandb_callback = wandb.keras.WandbCallback(
            save_model=False,
            compute_flops=True,
        )

tf.profiler.experimental.start('logdir')
history = model.fit(
    ds_train,
    epochs=epochs,
    validation_data=ds_val,
    callbacks=[tb_callback, wandb_callback],
)
# Train the model here
tf.profiler.experimental.stop()

2023-01-29 17:17:45.447871: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-01-29 17:17:45.447913: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2023-01-29 17:17:45.449079: I tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1664] Profiler found 1 GPUs
2023-01-29 17:17:45.475999: E tensorflow/core/profiler/backends/gpu/cupti_error_manager.cc:192] cuptiSubscribe: error 15: CUPTI_ERROR_NOT_INITIALIZED
2023-01-29 17:17:45.476043: E tensorflow/core/profiler/backends/gpu/cupti_error_manager.cc:457] cuptiGetResultString: ignored due to a previous error.
2023-01-29 17:17:45.476053: E tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1715] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error 
2023-01-29 17:17:45.476580: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.
2023-01-29 17:17:45.476656: E tensorflow/core/profi

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Epoch 1/20


/usr/local/lib/python3.8/dist-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-01-29 17:17:46.485896: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-01-29 17:17:47.272954: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-01-29 17:17:49.068884: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f7490027980 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-01-29 17:17:49.068930: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce GTX 165

      6/Unknown - 5s 11ms/step - loss: 4.5203 - accuracy: 0.0104  WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0077s vs `on_train_batch_end` time: 0.0080s). Check your callbacks.
ERROR:tensorflow:Failed to start profiler: Another profiler is running.
6977/6977 [==============================] - 87s 12ms/step - loss: 0.8793 - accuracy: 0.7108 - val_loss: 0.5382 - val_accuracy: 0.7993
Epoch 2/20
6977/6977 [==============================] - 83s 12ms/step - loss: 0.6205 - accuracy: 0.7782 - val_loss: 0.4918 - val_accuracy: 0.8138
Epoch 3/20
6977/6977 [==============================] - 83s 12ms/step - loss: 0.5770 - accuracy: 0.7904 - val_loss: 0.4683 - val_accuracy: 0.8218
Epoch 4/20
6977/6977 [==============================] - 87s 12ms/step - loss: 0.5558 - accuracy: 0.7964 - val_loss: 0.4534 - val_accuracy: 0.8265
Epoch 5/20
6977/6977 [==============================] - 86s 12ms/step - loss: 0.5409 - accuracy: 0.8008 - val_loss:

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
raise Exception("Stop right there criminal scum!")

In [ ]:
# calculate model size on disk, flops and number of parameters

flops = calculate_model_flops(wandb.run.summary)
disk_size = calculate_model_size_on_disk(f"./artifacts/{model_name}.h5")
num_parameters = calculate_model_num_parameters(model)

# evaluate model on ds_test and log to wandb
test_loss_before, test_acc_before = model.evaluate(ds_test)

wandb.log({
    "test loss": test_loss_before, 
    "test accuracy": test_acc_before, 
    "number of parameters": num_parameters,
    "disk size": disk_size, 
    "model flops": flops,
    })

# save base model to wandb
artifact = wandb.Artifact(
    name=model_name,
    type="model"
)

# save best model to artifact
artifact.add_file(f"./artifacts/{model_name}.h5")
run.log_artifact(artifact)


# pruning
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
num_images = get_number_of_examples(ds_train)
end_step = np.ceil(num_images / bs).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                            final_sparsity=0.80,
                                                            begin_step=0,
                                                            end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

callbacks = [
    wandb_callback,
    tfmot.sparsity.keras.UpdatePruningStep(),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=f"./artifacts/{model_name}_after.h5",
        save_weights_only=False,
        monitor="val_accuracy",
        mode="max",
        save_best_only=True,
    )
]


history = model_for_pruning.fit(
    ds_train,
    epochs=epochs,
    validation_data=ds_val,
    callbacks=callbacks,
)


# calculate model size on disk, flops and number of parameters

flops = calculate_model_flops(wandb.run.summary)
disk_size = calculate_model_size_on_disk(f"./artifacts/{model_name}_after.h5")
num_parameters = calculate_model_num_parameters(model)

# evaluate model on ds_test and log to wandb
test_loss_after, test_acc_after = model.evaluate(ds_test)

wandb.log({
    "test loss": test_loss_after, 
    "test accuracy": test_acc_after, 
    "number of parameters": num_parameters,
    "disk size": disk_size, 
    "model flops": flops,
    })


print('Baseline test accuracy:', test_acc_before) 
print('Pruned test accuracy:', test_acc_after)

# save base model to wandb
artifact = wandb.Artifact(
    name=f"{model_name}_after",
    type="model"
)

# save best model to artifact
artifact.add_file(f"./artifacts/{model_name}_after.h5")
run.log_artifact(artifact)

run.finish()

In [ ]:
defaults = dict(
    batch_size=32*2,
    epochs=30,    
    optimizer="adam"
)
train("baseline-pruned-numbers", defaults, artifact_name="numbers_splits_tfds")

In [ ]:
raise Exception("Stop right there!")

In [ ]:
train("baseline-pruned-all-characters", defaults, artifact_name="phcd_paper_splits_tfds")

In [ ]:
train("baseline-pruned-uppercase-diacritics", defaults, artifact_name="uppercase_splits_tfds")

In [ ]:
train("baseline-pruned-uppercase", defaults, artifact_name="uppercase_no_diacritics_splits_tfds")